# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [12]:
from tchub.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader

## Config parameters

Put here everything that could be needed if this notebook

In [13]:
config = AttrDict(
    use_wandb = False, # Whether to use or not wandb for experiment tracking
    wandb_group = 'embeddings', # Whether to group this run in a wandb group
    wandb_entity = 'pacmel',
    wandb_project = 'tchub',
    enc_artifact = 'pacmel/tchub/dcae:run-tchub-1j68l02a', # Name:version of the encoder artifact
    input_ar = None,
    cpu = False
)

## Run

In [14]:
run = wandb.init(entity=config.wandb_entity,
                    project=config.wandb_project if config.use_wandb else 'work-nbs', 
                    group=config.wandb_group,
                    job_type='embeddings', 
                    mode='online' if config.use_wandb else 'disabled',
                    anonymous = 'never' if config.use_wandb else 'must',
                    config=config,
                    #id = 'embeddingsProvider',
                    resume='allow')

In [15]:
# Botch to use artifacts offline
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

Restore the encoder model and its associated configuration

In [16]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [17]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()
enc_learner

AttributeError: Can't get attribute 'Conv1dSame' on <module 'tsai.models.layers' from '/home/victor/lib/tsai/tsai/models/layers.py'>

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [ ]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_valid = artifacts_gettr(enc_run.config['valid_artifact'], type='dataset')
enc_artifact_train.name, enc_artifact_valid.name

Now we specify the dataset artifact that we want to get the embeddings from. If no 
artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [ ]:
input_ar_name = ifnone(config.input_ar, 
                       f'{enc_artifact_valid.entity}/{enc_artifact_valid.project}/{enc_artifact_valid.name}')
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

In [ ]:
df = input_ar.to_df()
df

In [ ]:
df.shape

In [ ]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

In [ ]:
%%time
embs = get_enc_embs(enc_input, enc_learner, cpu=config.cpu, to_numpy=True)

In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(embs, 'embeddings', metadata=dict(run.config)), 
                     aliases=f'run-{run.project}-{run.id}')

In [ ]:
run.finish()